# Understanding the Image Class: Data Components and Accessors

This tutorial provides a comprehensive guide to the PhenoTypic `Image` class architecture, focusing on its main data components and how they interact with detection and analysis modules.

## Learning Objectives

By the end of this tutorial, you will understand:
- The Image class hierarchy and accessor pattern
- The six main data components: `rgb`, `gray`, `enh_gray`, `objmask`, `objmap`, and `metadata`
- How these components interact during image processing workflows
- When and how to use each component in your analysis


## 1. Introduction: Image Class Architecture

### Class Hierarchy

The `Image` class inherits from several specialized classes, each adding specific functionality:

```
Image
  ↓
ImageIOHandler (file I/O operations)
  ↓
ImageColorSpace (color space conversions)
  ↓
...(additional specialized handlers)
  ↓
ImageHandler (accessor initialization and image operations)
  ↓
ImageDataManager (core data storage and management)
```

### The Accessor Pattern

PhenoTypic uses an **accessor pattern** to provide intuitive, property-based access to image data. Instead of directly accessing internal arrays, you use accessor objects that:
- Provide consistent interfaces across different data types
- Handle synchronization between related data (e.g., mask and map)
- Support NumPy-style indexing with bracket notation: `image.rgb[:]`
- Enable method chaining and specialized operations

### Internal Data Storage

The Image class stores data in two main containers:

1. **`_data` (ImageData)**: Holds the actual image arrays
   - `rgb`: NumPy array (uint8/uint16) for color images
   - `gray`: NumPy array (float32) for grayscale representation
   - `enh_gray`: NumPy array (float32) for enhanced grayscale
   - `sparse_object_map`: Sparse CSC matrix (uint16) for labeled objects

2. **`_metadata` (Metadata)**: Stores image information in three tiers
   - `private`: Internal use only (e.g., UUID)
   - `protected`: System-managed (e.g., name, bit_depth, image_type)
   - `public`: User-modifiable custom metadata


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import phenotypic as pht
from phenotypic.detect import OtsuDetector

# Set matplotlib style for better visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100


## 2. Loading an Image and Basic Properties

Let's start by loading a sample plate image from the built-in dataset. We'll use an image of bacterial colonies growing on agar media.


In [ ]:
# Load a sample plate image (12 hours of growth)
image = pht.data.load_plate_12hr(mode='Image')

# Display basic properties
print(f"Image name: {image.name}")
print(f"Image shape: {image.shape}")
print(f"Bit depth: {image.bit_depth}")
print(f"Data type: {type(image)}")


In [ ]:
# Visualize the original image
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
image.show(ax=ax)
ax.set_title(f'{image.name} - Original Image', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## 3. RGB Component: Multichannel Color Data

The **`rgb`** accessor provides access to the multichannel RGB representation of the image.

### Key Characteristics:
- **Data type**: uint8 (for 8-bit) or uint16 (for 16-bit images)
- **Shape**: `(height, width, 3)` - three channels for Red, Green, Blue
- **Access pattern**: Use bracket notation `image.rgb[:]` to get the array
- **Automatically set**: When loading a color image
- **Empty for grayscale**: If you load a 2D array, RGB remains empty


In [ ]:
# Access the RGB data
rgb_data = image.rgb[:]

print(f"RGB shape: {rgb_data.shape}")
print(f"RGB dtype: {rgb_data.dtype}")
print(f"RGB value range: [{rgb_data.min()}, {rgb_data.max()}]")
print(f"\nIs RGB empty? {image.rgb.isempty()}")


In [ ]:
# Visualize RGB and individual channels
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Full RGB
axes[0, 0].imshow(rgb_data)
axes[0, 0].set_title('RGB (Full Color)', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Red channel
axes[0, 1].imshow(rgb_data[:, :, 0], cmap='Reds')
axes[0, 1].set_title('Red Channel', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

# Green channel
axes[1, 0].imshow(rgb_data[:, :, 1], cmap='Greens')
axes[1, 0].set_title('Green Channel', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

# Blue channel
axes[1, 1].imshow(rgb_data[:, :, 2], cmap='Blues')
axes[1, 1].set_title('Blue Channel', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()


## 4. Gray Component: Grayscale Representation

The **`gray`** accessor provides the grayscale representation of the image using weighted luminance conversion.

### Key Characteristics:
- **Data type**: float32 with values in [0, 1] range
- **Shape**: `(height, width)` - single channel
- **Conversion**: Automatically created from RGB using `skimage.color.rgb2gray`
- **Weighted formula**: `0.2125*R + 0.7154*G + 0.0721*B` (ITU-R BT.709)
- **Read-only**: Changes to gray won't modify RGB (use `enh_gray` for modifications)
- **Always present**: Even for grayscale-loaded images


In [ ]:
# Access the grayscale data
gray_data = image.gray[:]

print(f"Gray shape: {gray_data.shape}")
print(f"Gray dtype: {gray_data.dtype}")
print(f"Gray value range: [{gray_data.min():.4f}, {gray_data.max():.4f}]")
print(f"\nIs gray empty? {image.gray.isempty()}")


In [ ]:
# Visualize grayscale conversion
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Original RGB
axes[0].imshow(rgb_data)
axes[0].set_title('Original RGB Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Grayscale
axes[1].imshow(gray_data, cmap='gray')
axes[1].set_title('Grayscale (Weighted Luminance)', fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Note: The grayscale conversion preserves perceptual brightness better than simple averaging.")


## 5. Enhanced Gray Component: Mutable Processing Copy

The **`enh_gray`** accessor provides a mutable copy of the grayscale image for preprocessing and enhancement.

### Key Characteristics:
- **Data type**: float32 with values in [0, 1] range
- **Shape**: `(height, width)` - single channel
- **Initialization**: Starts as an exact copy of `gray`
- **Mutable**: Can be modified without affecting the original `gray`
- **Primary use**: Detection algorithms process `enh_gray`, preserving original data
- **Workflow**: Apply enhancement operations → Detector uses `enh_gray` → Original stays intact


In [ ]:
# Access enhanced grayscale data
enh_gray_data = image.enh_gray[:]

print(f"Enhanced gray shape: {enh_gray_data.shape}")
print(f"Enhanced gray dtype: {enh_gray_data.dtype}")
print(f"Enhanced gray value range: [{enh_gray_data.min():.4f}, {enh_gray_data.max():.4f}]")

# Check if enh_gray starts as a copy of gray
print(f"\nAre gray and enh_gray initially equal? {np.allclose(gray_data, enh_gray_data)}")
print(f"Are they the same object? {np.shares_memory(gray_data, enh_gray_data)}")


In [ ]:
# Demonstrate that enh_gray can be modified independently
# Let's apply a simple enhancement (histogram equalization)
from skimage.exposure import equalize_adapthist

# Create a copy to show the enhancement
enhanced_demo = equalize_adapthist(enh_gray_data, clip_limit=0.03)

# Visualize the difference
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(gray_data, cmap='gray')
axes[0].set_title('Gray (Original)', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(enh_gray_data, cmap='gray')
axes[1].set_title('Enh_Gray (Initial Copy)', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(enhanced_demo, cmap='gray')
axes[2].set_title('Enh_Gray (After CLAHE)', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Note: Enhancement operations can be applied to enh_gray without affecting the original gray data.")


## 6. Object Mask and Object Map: Detection Results

The **`objmask`** and **`objmap`** accessors store the results of object detection algorithms. Let's apply the `OtsuDetector` to see them in action.

### OtsuDetector Workflow:
1. Reads `enh_gray` data
2. Calculates optimal threshold using Otsu's method
3. Creates binary mask and stores in `objmask`
4. Labels connected regions and stores in `objmap`


In [ ]:
# Apply OtsuDetector to detect colonies
detector = OtsuDetector(ignore_zeros=True, ignore_borders=True)
detector.apply(image)

print("✅ Detection complete!")
print(f"Number of detected objects: {image.num_objects}")


### 6.1 Object Mask: Binary Representation

The **`objmask`** is a binary mask showing which pixels belong to detected objects.

### Key Characteristics:
- **Values**: 0 (background) and 1 (object pixels)
- **Shape**: `(height, width)` - matches image dimensions
- **Storage**: Backed by sparse CSC matrix for memory efficiency
- **Mutable**: Can be modified (triggers automatic objmap relabeling)
- **Set by**: Detection algorithms (Otsu, Watershed, etc.)


In [ ]:
# Access object mask
objmask_data = image.objmask[:]

print(f"Object mask shape: {objmask_data.shape}")
print(f"Object mask dtype: {objmask_data.dtype}")
print(f"Unique values: {np.unique(objmask_data)}")
print(f"Number of object pixels: {np.sum(objmask_data)}")
print(f"Number of background pixels: {np.sum(objmask_data == 0)}")
print(f"Percentage of image covered by objects: {100 * np.sum(objmask_data) / objmask_data.size:.2f}%")


In [ ]:
# Visualize the object mask
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Original grayscale
axes[0].imshow(gray_data, cmap='gray')
axes[0].set_title('Original Grayscale', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Binary mask
axes[1].imshow(objmask_data, cmap='binary')
axes[1].set_title('Object Mask (Binary)', fontsize=12, fontweight='bold')
axes[1].axis('off')

# Overlay on original
axes[2].imshow(gray_data, cmap='gray')
axes[2].imshow(objmask_data, cmap='Reds', alpha=0.4)
axes[2].set_title('Mask Overlay on Image', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Note: The binary mask shows all detected object pixels without distinguishing individual objects.")


### 6.2 Object Map: Labeled Regions

The **`objmap`** assigns a unique integer ID to each connected region in the object mask.

### Key Characteristics:
- **Values**: 0 (background) and unique integers for each object (1, 2, 3, ...)
- **Shape**: `(height, width)` - matches image dimensions
- **Storage**: Sparse CSC matrix (shared backend with objmask)
- **Automatic labeling**: Created/updated when objmask changes
- **Used for**: Individual object analysis, measurements, slicing


In [ ]:
# Access object map
objmap_data = image.objmap[:]

print(f"Object map shape: {objmap_data.shape}")
print(f"Object map dtype: {objmap_data.dtype}")
print(f"Number of unique objects: {len(np.unique(objmap_data)) - 1}")
print(f"Object IDs range: {objmap_data[objmap_data > 0].min()} to {objmap_data.max()}")
print(f"\nFirst 10 object IDs: {sorted(np.unique(objmap_data[objmap_data > 0]))[:10]}")


In [ ]:
# Visualize the labeled object map
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Object mask (binary)
axes[0].imshow(objmask_data, cmap='binary')
axes[0].set_title(f'Object Mask\n(Binary: {image.num_objects} objects)', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Object map (labeled)
axes[1].imshow(objmap_data, cmap='nipy_spectral')
axes[1].set_title('Object Map\n(Each color = unique object)', fontsize=12, fontweight='bold')
axes[1].axis('off')

# Overlay on original
axes[2].imshow(rgb_data)
# Create a masked array to show only objects
masked_objmap = np.ma.masked_where(objmap_data == 0, objmap_data)
axes[2].imshow(masked_objmap, cmap='nipy_spectral', alpha=0.5, interpolation='none')
axes[2].set_title('Labeled Objects Overlay', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Note: Each color in the object map represents a distinct detected colony.")


### Relationship Between objmask and objmap

The objmask and objmap are tightly coupled:
- **Shared backend**: Both use the same sparse CSC matrix storage
- **objmask → objmap**: Binary mask shows "where objects are"
- **objmap → objmask**: Labeled map shows "which object each pixel belongs to"
- **Automatic sync**: Modifying objmask triggers relabeling of objmap


In [ ]:
# Demonstrate the relationship
print("Relationship verification:")
print(f"objmask non-zero pixels: {np.count_nonzero(objmask_data)}")
print(f"objmap non-zero pixels: {np.count_nonzero(objmap_data)}")
print(f"Are non-zero locations identical? {np.all((objmask_data > 0) == (objmap_data > 0))}")

print(f"\nobjmask values: {np.unique(objmask_data)}")
print(f"objmap values: {len(np.unique(objmap_data))} unique (0 + {image.num_objects} objects)")


## 7. Metadata: Image Information Storage

The **`metadata`** accessor provides a three-tier system for storing image information.

### Three-Tier System:

1. **Private**: Internal system use only
   - UUID for unique identification
   - Not directly modifiable by users

2. **Protected**: System-managed metadata
   - `image_name`: Image identifier
   - `bit_depth`: 8 or 16 bit
   - `image_type`: BASE, OBJECT, or GRID
   - Can be read but modification is restricted

3. **Public**: User-modifiable metadata
   - Custom key-value pairs
   - Experimental conditions, timestamps, etc.
   - Fully modifiable by users


In [ ]:
# Access metadata
print("=" * 50)
print("METADATA CONTENTS")
print("=" * 50)

print("\nAll available metadata keys:")
for key in image.metadata.keys():
    print(f"  - {key}")

print("\n" + "-" * 50)
print("Key metadata values:")
print("-" * 50)
print(f"Name: {image.metadata['image_name']}")
print(f"Bit depth: {image.metadata['bit_depth']}")
print(f"Image type: {image.metadata['image_type']}")


In [ ]:
# Add custom metadata (public tier)
image.metadata['experiment_id'] = 'EXP_001'
image.metadata['growth_time_hours'] = 12
image.metadata['strain'] = 'E. coli K-12'
image.metadata['temperature_celsius'] = 37.0
image.metadata['media_type'] = 'LB Agar'

print("✅ Custom metadata added!\n")

# Display all metadata
print("Updated metadata:")
for key, value in image.metadata.items():
    print(f"  {key}: {value}")


## 8. Component Interactions: Complete Workflow

Let's visualize how all components work together in a typical image processing workflow.


In [ ]:
# Create a comprehensive visualization showing all components
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Row 1: Source data
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(image.rgb[:])
ax1.set_title('1. RGB Component\n(Color image data)', fontsize=11, fontweight='bold')
ax1.axis('off')

ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(image.gray[:], cmap='gray')
ax2.set_title('2. Gray Component\n(Luminance conversion)', fontsize=11, fontweight='bold')
ax2.axis('off')

ax3 = fig.add_subplot(gs[0, 2])
ax3.imshow(image.enh_gray[:], cmap='gray')
ax3.set_title('3. Enh_Gray Component\n(Processing copy)', fontsize=11, fontweight='bold')
ax3.axis('off')

# Row 2: Detection results
ax4 = fig.add_subplot(gs[1, 0])
ax4.imshow(image.objmask[:], cmap='binary')
ax4.set_title(f'4. ObjMask Component\n(Binary mask: {image.num_objects} objects)', fontsize=11, fontweight='bold')
ax4.axis('off')

ax5 = fig.add_subplot(gs[1, 1])
ax5.imshow(image.objmap[:], cmap='nipy_spectral')
ax5.set_title('5. ObjMap Component\n(Labeled regions)', fontsize=11, fontweight='bold')
ax5.axis('off')

ax6 = fig.add_subplot(gs[1, 2])
# Create text display for metadata
ax6.axis('off')
metadata_text = "6. Metadata Component\n" + "="*25 + "\n"
metadata_text += f"Name: {image.metadata.get('image_name', 'N/A')}\n"
metadata_text += f"Bit depth: {image.metadata.get('bit_depth', 'N/A')}\n"
metadata_text += f"Type: {image.metadata.get('image_type', 'N/A')}\n"
metadata_text += f"Experiment: {image.metadata.get('experiment_id', 'N/A')}\n"
metadata_text += f"Strain: {image.metadata.get('strain', 'N/A')}\n"
metadata_text += f"Growth time: {image.metadata.get('growth_time_hours', 'N/A')} hrs\n"
ax6.text(0.1, 0.5, metadata_text, fontsize=10, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax6.set_title('Metadata', fontsize=11, fontweight='bold')

# Row 3: Overlays
ax7 = fig.add_subplot(gs[2, :])
ax7.imshow(image.rgb[:])
masked_objmap = np.ma.masked_where(image.objmap[:] == 0, image.objmap[:])
ax7.imshow(masked_objmap, cmap='nipy_spectral', alpha=0.5, interpolation='none')
ax7.set_title(f'Complete Workflow Result: {image.num_objects} Detected Colonies', 
              fontsize=12, fontweight='bold')
ax7.axis('off')

plt.suptitle('PhenoTypic Image Class: All Data Components', fontsize=16, fontweight='bold', y=0.98)
plt.show()


### Workflow Summary

```
┌─────────────────────────────────────────────────────────────────┐
│ 1. Load RGB Image (from file or array)                         │
│    → image = pht.Image.imread('plate.jpg')                      │
└────────────────────┬────────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────────┐
│ 2. Auto-convert to Gray (weighted luminance)                   │
│    → image.gray[:] available immediately                        │
└────────────────────┬────────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────────┐
│ 3. Initialize Enh_Gray (copy of gray)                          │
│    → image.enh_gray[:] = image.gray[:].copy()                  │
└────────────────────┬────────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────────┐
│ 4. Apply Detector (operates on enh_gray)                       │
│    → detector.apply(image)                                      │
└────────────────────┬────────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────────┐
│ 5. Detector Sets ObjMask (binary detection result)             │
│    → image.objmask[:] = binary_mask                             │
└────────────────────┬────────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────────┐
│ 6. Auto-label ObjMap (connected component labeling)            │
│    → image.objmap[:] automatically updated                      │
└─────────────────────────────────────────────────────────────────┘
```


## 9. Practical Tips and Best Practices

### When to Use Each Component

| Component | Use When You Need To... |
|-----------|------------------------|
| **rgb** | Access color information, display full-color images, analyze individual color channels |
| **gray** | Work with intensity data, maintain original data integrity, perform measurements |
| **enh_gray** | Apply preprocessing, test enhancement algorithms, prepare for detection |
| **objmask** | Work with binary masks, perform morphological operations, modify detection results |
| **objmap** | Analyze individual objects, measure per-object properties, extract specific colonies |
| **metadata** | Store experimental context, track processing parameters, organize datasets |

### Accessor Pattern Benefits

1. **Consistent Interface**: All accessors use bracket notation `[:]`
2. **Data Integrity**: Read-only accessors prevent accidental modifications
3. **Automatic Sync**: Changes to objmask automatically update objmap
4. **Memory Efficiency**: Sparse storage for object maps saves memory
5. **Type Safety**: Accessors enforce correct data types

### Read-Only vs Mutable Components

**Read-Only** (viewing via `[:]` returns non-writable views):
- `rgb`: Preserve color data integrity
- `gray`: Maintain original grayscale reference

**Mutable** (can be modified in-place):
- `enh_gray`: Designed for preprocessing
- `objmask`: Can be refined post-detection
- `objmap`: Automatically updates with objmask
- `metadata`: Fully modifiable for custom data

### Integration with Other PhenoTypic Modules

**Detection** (`phenotypic.detect`):
```python
from phenotypic.detect import OtsuDetector, WatershedDetector
detector = OtsuDetector()
detector.apply(image)  # Populates objmask and objmap
```

**Enhancement** (`phenotypic.enhance`):
```python
from phenotypic.enhance import CLAHE, GaussianBlur
enhancer = CLAHE()
enhancer.apply(image)  # Modifies enh_gray
```

**Measurement** (`phenotypic.measure`):
```python
from phenotypic.measure import MeasureSize
measurer = MeasureSize()
results = measurer.apply(image)  # Uses objmap for per-object analysis
```

**Analysis** (`phenotypic.analysis`):
```python
# Access individual objects using objmap labels
for obj in image.objects:
    print(f"Object {obj.label}: Area = {obj.area}")
```


## Key Takeaways

### Architecture
✅ The Image class uses a layered inheritance structure with specialized handlers

✅ The accessor pattern provides intuitive, property-based access to image data

✅ Internal storage uses `_data` (ImageData) and `_metadata` (Metadata) containers

### Data Components
✅ **rgb**: Color image data (uint8/uint16, 3 channels)

✅ **gray**: Grayscale representation (float32, auto-converted via weighted luminance)

✅ **enh_gray**: Mutable processing copy (float32, for preprocessing/detection)

✅ **objmask**: Binary object mask (0/1, shows detected pixels)

✅ **objmap**: Labeled object regions (0/1/2/3..., unique ID per object)

✅ **metadata**: Three-tier information storage (private/protected/public)

### Workflow
✅ Loading → Gray conversion → Enh_gray initialization → Detection → Mask creation → Map labeling

✅ Components are synchronized automatically (objmask ↔ objmap)

✅ Original data remains intact during processing (gray vs enh_gray separation)

### Best Practices
✅ Use bracket notation `[:]` to access array data from accessors

✅ Modify `enh_gray` for preprocessing, keep `gray` as reference

✅ Store experimental metadata in the public tier for reproducibility

✅ Use objmap for individual object analysis, objmask for morphological operations


## Next Steps

Now that you understand the Image class structure, explore:

1. **Grid Images**: Specialized subclass for arrayed colonies (`GridImage`)
2. **Image Sets**: Batch processing multiple images (`ImageSet`)
3. **Detection Algorithms**: Various threshold and watershed methods
4. **Enhancement Operations**: CLAHE, filtering, morphological operations
5. **Measurement Tools**: Size, intensity, shape, texture analysis
6. **Interactive Tools**: Jupyter-based interactive measurement (`image.interactive_measure()`)

Check out the other example notebooks for more advanced workflows!
